In [ ]:
%pip install scikit-image --quiet
%pip install numpy --quiet
%pip install matplotlib --quiet
%pip install scipy --quiet
%pip install opencv-python --quiet
%pip install opencv-python-headless --quiet
%pip install opencv-contrib-python --quiet
%pip install scikit-learn --quiet
%pip install seaborn --quiet

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import io
from skimage import color
from skimage import filters

In [ ]:
# Config
ORB_FEATURES = 1000 # Number of features to extract
KNN = 3 # Number of neighbors to use in KNN

ADD_VARIATION = True # Add variations to the dataset
ADD_ROTATION = True # Add rotated images to the dataset
ADD_FLIP = True # Add flipped images to the dataset
ADD_SCALE = True # Add scaled images to the dataset

data_train_dir = 'data/train' # Directory containing training data

# Directory containing handpicked examples that can stand as outliers
# These are used to do further testing on the model,
# besides the standard train/test split
data_test_dir = 'data/test' 


In [ ]:
# Load the images and keep the folder name as the label
def load_images(folder, n=None):
    images = []
    labels = []
    for label in os.listdir(folder):
        for filename in os.listdir(os.path.join(folder, label)):
            img = cv2.imread(os.path.join(folder, label, filename))
            if img is not None:
                images.append(img)
                labels.append(label)
            
            if n is not None and len(images) >= n:
                break
    return images, labels

images, labels = load_images(data_train_dir)

In [ ]:
# ensure that there are no duplicates // count
for i in range(len(images)):
    for j in range(i+1, len(images)):
        if np.array_equal(images[i], images[j]):
            print(i, j)
            print(labels[i], labels[j])
            print(np.array_equal(images[i], images[j]))
            print('---')

In [ ]:
# helper function to display images
def display( l_images, r_images, l_labels,r_labels=None, display_images=None):
    if r_labels is None:
        r_labels = l_labels
    if display_images is None:
        display_images = range(len(l_images))
    for i in display_images:
        f, axarr = plt.subplots(1, 2, figsize=(10, 5))
        axarr[0].imshow(cv2.cvtColor(l_images[i], cv2.COLOR_BGR2RGB))
        axarr[0].set_title(str(i+1) + ': ' + l_labels[i])
        axarr[0].axis('off')
        axarr[1].imshow(cv2.cvtColor(r_images[i], cv2.COLOR_BGR2RGB))
        axarr[1].set_title(str(i+1) + ': ' + r_labels[i] + ' (processed)')
        axarr[1].axis('off')
        plt.show()
display(images, images, labels, labels, [0])

In [ ]:
# Add variations to the dataset
def variants(img):
    if not ADD_VARIATION:
        return [img]
    variants = []
    angles = [0, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE]
    flips = [0, 1]
    diagonal_angles = [45, 135, 225, 315]
    for angle in angles:
        var = cv2.rotate(img, angle)
        variants.append(var)
        for flip in flips:
            var = cv2.flip(var, flip)
            variants.append(var)
    
    # add rotated images
    for angle in diagonal_angles:
        M = cv2.getRotationMatrix2D((img.shape[1]//2, img.shape[0]//2), angle, 1)
        var = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), borderValue=(255, 255, 255))
        variants.append(var)
        for flip in flips:
            var = cv2.flip(var, flip)
            variants.append(var)
    
    # rm duplicates
    new_variants = []
    for i in range(len(variants)):
        for j in range(0, len(new_variants)):
            if np.array_equal(variants[i], new_variants[j]):
                break
        else:
            new_variants.append(variants[i])
    return new_variants



variant_images = []
variant_labels = []
for i in range(len(images)):
    for img in variants(images[i]):
        variant_images.append(img)
        variant_labels.append(labels[i])

# plot the first image and its variants in a small grid
plt.figure(figsize=(10, 10))
for i in range(20):
    plt.subplot(5, 5, i+1)
    plt.imshow(cv2.cvtColor(variant_images[i], cv2.COLOR_BGR2RGB))
    plt.title(variant_labels[i])
    plt.axis('off')
plt.show()

In [ ]:
import math
# Preprocess the images
def preprocess(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    img = cv2.GaussianBlur(img, (5, 5), 0)
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    kernel = np.ones((3,3),np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return img

preprocessed_images = [preprocess(img) for img in variant_images]

display(variant_images, preprocessed_images, variant_labels, variant_labels, [0, 426, 400])


In [ ]:
# align the images
def contour_segmentation(img):
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # find the contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # find the largest contour
    max_area = 0
    max_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_contour = contour
    # find the bounding rectangle
    x, y, w, h = cv2.boundingRect(max_contour)
    # crop the image  into rect
    img = img[y:y+h, x:x+w]
    # img = img[y:y+(h if h < w else w), x:x+(h if h < w else w)]
    return img

def grab_cut(img):

    # remove alpha channel
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

    # create a mask
    mask = np.zeros(img.shape[:2], np.uint8)

    # define the background and foreground models
    bgdModel = np.zeros((1, 65), np.float64)

    fgdModel = np.zeros((1, 65), np.float64)

    # define the rectangle
    perc = 0.1
    padX = int(perc * img.shape[0])
    padY = int(perc * img.shape[1])

    rect = (padX, padY, img.shape[1] - 2 * padX, img.shape[0] - 2 * padY)
    # apply grabcut
    cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

    # create a mask with the probable foreground
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    # apply the mask to the image
    img = img * mask2[:, :, np.newaxis]

    return img

aligned_images = [contour_segmentation(img) for img in preprocessed_images]
# aligned_images = [grab_cut(img) for img in preprocessed_images]
display(preprocessed_images, aligned_images, variant_labels, variant_labels, [0, 426, 400])

In [ ]:
# orientation so that the longest side is the width
def orientation(img):
    # return img if img.shape[0] < img.shape[1] else cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    return img

oriented = [orientation(img) for img in aligned_images]

display(aligned_images, oriented, variant_labels, variant_labels, [0, 426, 400])

In [ ]:
# Feature extraction
def extract_features_orb(img, n_features=ORB_FEATURES):
    orb = cv2.ORB_create(n_features)
    keypoints, descriptors = orb.detectAndCompute(img, None)
    return keypoints, descriptors

def extract_features_sift(img):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    return keypoints, descriptors

def extract_features_harris(img):
    # remove alpha channel
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray, 2, 3, 0.04)
    dst = cv2.dilate(dst, None)
    ret, dst = cv2.threshold(dst, 0.01 * dst.max(), 255, 0)
    dst = np.uint8(dst)
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray, np.float32(centroids), (5, 5), (-1, -1), criteria)
    return corners

def extract_features_hog(img):
    hog = cv2.HOGDescriptor()
    h = hog.compute(img)
    return h

## ------------------------------------------------------------------------------------------------
## CHANGE THIS TO USE THE FEATURE EXTRACTION METHOD YOU WANT
# keypoints, descriptors = zip(*[extract_features_orb(img) for img in oriented])
# keypoints, descriptors = zip(*[extract_features_sift(img) for img in oriented])
descriptors = [extract_features_harris(img) for img in oriented]
# keypoints = [extract_features_hog(img) for img in oriented]
## ------------------------------------------------------------------------------------------------


# check if any features were found
for i in range(len(descriptors)):
    if descriptors[i] is None:
        print('No features found for image', i)
        plt.imshow(cv2.cvtColor(aligned_images[i], cv2.COLOR_BGR2RGB))
        plt.title('No features found')
        plt.axis('off')
        plt.show()

print('Number of descriptors:',len(descriptors))
print('All shapes:', [d.shape for d in descriptors])

In [ ]:
# display keypoints
def display_keypoints(img, keypoints):
    img = cv2.drawKeypoints(img, keypoints, None)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

display_keypoints(oriented[0], keypoints[0])
display_keypoints(oriented[25], keypoints[25])

In [ ]:
from sklearn.cluster import MiniBatchKMeans

kmeans = MiniBatchKMeans(n_clusters=1000, random_state=42)
kmeans.fit(np.vstack(descriptors))

# save the kmeans model
import pickle
with open('kmeans.pkl', 'wb') as f:
    pickle.dump(kmeans, f)


In [ ]:
# bag of visual words
def bag_of_words(descriptors, kmeans):
    histogram = np.zeros(kmeans.n_clusters)
    if descriptors is not None:
        clusters = kmeans.predict(descriptors)
        for cluster in clusters:
            histogram[cluster] += 1
    return histogram

bovw = np.array([bag_of_words(d, kmeans) for d in descriptors])
bovw.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(bovw, variant_labels, test_size=0.2, random_state=42, stratify=variant_labels)
X_train.shape, X_test.shape

In [ ]:
# RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)





import seaborn as sns
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=rf.classes_, yticklabels=rf.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# save the model
import joblib
joblib.dump(rf, 'rf.pkl')




In [ ]:
# SVM
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=svm.classes_, yticklabels=svm.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# save the model
joblib.dump(svm, 'svc.pkl')

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=knn.classes_, yticklabels=knn.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# save the model
joblib.dump(knn, 'knn.pkl')


In [ ]:
# additional more complex testing
# some very specific examples that are difficult to classify
test_images, test_labels = load_images(data_test_dir)
print(test_labels)
preprocessed_test_images = [preprocess(img) for img in test_images]
aligned_test_images = [contour_segmentation(img) for img in preprocessed_test_images]
oriented_test_images = [orientation(img) for img in aligned_test_images]
# keypoints_test, descriptors_test = zip(*[extract_features_orb(img) for img in oriented_test_images])
# keypoints_test, descriptors_test = zip(*[extract_features_sift(img) for img in oriented_test_images])
descriptors_test = [extract_features_harris(img) for img in oriented_test_images]
bovw_test = np.array([bag_of_words(d, kmeans) for d in descriptors_test])
y_pred = rf.predict(bovw_test)
print('Accuracy:', accuracy_score(test_labels, y_pred))

y_pred = svm.predict(bovw_test)
print('Accuracy:', accuracy_score(test_labels, y_pred))

y_pred = knn.predict(bovw_test)
print('Accuracy:', accuracy_score(test_labels, y_pred))

# print oriented_test_images
display(preprocessed_test_images, oriented_test_images, test_labels, y_pred)